In [7]:
import json
import glob
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

In [8]:
def prep_json(d):
    d = d['results'][0]
    if sum(d['exit_codes']) != 0:
        raise ValueError('exit_codes != 0')
    d = {'num_ifs': params[0],
        'num_threads': params[1],
        'array_size': params[2],
        'n': len(d['times'])} | d
    d['ci'] = 1.960 * d['stddev'] / (d['n']**0.5)
    _ = d.pop('command')
    _ = d.pop('times')
    _ = d.pop('exit_codes')
    return d

In [9]:
out = 'out/20220731101132/'
data = []
for fpath in glob.iglob(out+'hf/*'):
    fname, fext = fpath.split('/')[-1].split('.')
    if fext == 'json':
        params = [int(x) for x in fname.split('_')]
        with open(fpath, 'r') as f:
            data.append(prep_json(json.load(f)))
df = pd.DataFrame(data).sort_values(['num_ifs','num_threads','array_size'])
df

,num_ifs,num_threads,array_size,n,mean,stddev,median,user,system,min,max,ci
198,0,1,1,100,0.002824,0.000227,0.002852,0.000968,0.000871,0.002248,0.003385,0.000044
155,0,1,10,100,0.002715,0.000210,0.002729,0.000932,0.000755,0.002178,0.003462,0.000041
495,0,1,100,100,0.002754,0.000221,0.002768,0.000973,0.000770,0.002219,0.003369,0.000043
383,0,1,1000,100,0.003173,0.000190,0.003188,0.001369,0.000766,0.002706,0.003788,0.000037
310,0,1,10000,100,0.007088,0.000187,0.007080,0.005165,0.000793,0.006622,0.007852,0.000037
...,...,...,...,...,...,...,...,...,...,...,...,...
219,10000,4096,100,100,0.193100,0.002592,0.193009,0.028526,0.318859,0.186807,0.199367,0.000508
443,10000,4096,1000,100,0.194090,0.002379,0.193790,0.029979,0.321537,0.187868,0.201080,0.000466
327,10000,4096,10000,100,0.197888,0.003360,0.197353,0.037236,0.326702,0.191389,0.210862,0.000658
212,10000,4096,100000,100,0.203358,0.003205,0.203274,0.077805,0.346546,0.196746,0.214141,0.000628


In [10]:
def line(error_y_mode=None, **kwargs):
    """Extension of `plotly.express.line` to use error bands.
    Source: https://stackoverflow.com/a/69594497"""
    ERROR_MODES = {'bar','band','bars','bands',None}
    if error_y_mode not in ERROR_MODES:
        raise ValueError(f"'error_y_mode' must be one of {ERROR_MODES}, received {repr(error_y_mode)}.")
    if error_y_mode in {'bar','bars',None}:
        fig = px.line(**kwargs)
    elif error_y_mode in {'band','bands'}:
        if 'error_y' not in kwargs:
            raise ValueError(f"If you provide argument 'error_y_mode' you must also provide 'error_y'.")
        figure_with_error_bars = px.line(**kwargs)
        fig = px.line(**{arg: val for arg,val in kwargs.items() if arg != 'error_y'})
        for data in figure_with_error_bars.data:
            x = list(data['x'])
            y_upper = list(data['y'] + data['error_y']['array'])
            y_lower = list(data['y'] - data['error_y']['array'] if data['error_y']['arrayminus'] is None else data['y'] - data['error_y']['arrayminus'])
            color = f"rgba({tuple(int(data['line']['color'].lstrip('#')[i:i+2], 16) for i in (0, 2, 4))},.3)".replace('((','(').replace('),',',').replace(' ','')
            fig.add_trace(
                go.Scatter(
                    x = x+x[::-1],
                    y = y_upper+y_lower[::-1],
                    fill = 'toself',
                    fillcolor = color,
                    line = dict(
                        color = 'rgba(255,255,255,0)'
                    ),
                    hoverinfo = "skip",
                    showlegend = False,
                    legendgroup = data['legendgroup'],
                    xaxis = data['xaxis'],
                    yaxis = data['yaxis'],
                )
            )
        # Reorder data as said here: https://stackoverflow.com/a/66854398/8849755
        reordered_data = []
        for i in range(int(len(fig.data)/2)):
            reordered_data.append(fig.data[i+int(len(fig.data)/2)])
            reordered_data.append(fig.data[i])
        fig.data = tuple(reordered_data)
    return fig

In [11]:
def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

n_colors = 13
colors = px.colors.sample_colorscale(
    'plasma_r', 
    [n/(n_colors -1) for n in range(n_colors)])
colors = ['#'+rgb_to_hex(eval(c.lstrip('rgb'))) for c in colors]
colors

['#f0f921',
 '#fad625',
 '#fcb430',
 '#f89640',
 '#ed7953',
 '#dd5f65',
 '#cb4778',
 '#b52f8c',
 '#9c179e',
 '#7c06a6',
 '#5c02a3',
 '#380499',
 '#0d0887']

In [12]:
for array_size in df['array_size'].unique():
    fig = line(
        data_frame=df[df['array_size']==array_size],
        title=f'Array size = {array_size}',
        x='num_ifs',
        y='mean',
        log_y=True,
        error_y = 'ci',
        error_y_mode = 'band',
        color='num_threads',
        color_discrete_sequence=colors)
    fig.update_xaxes(type='category')
    fig.update_layout(margin=dict(l=0, r=120, t=50, b=0))
    fig.write_image(out+f'{array_size}.png', scale=2)
    fig.show()